In [62]:
import numpy as np
import pandas as pd

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import numpy as np
with open('/content/drive/MyDrive/reviews.txt', 'r') as f:
    reviews = f.read()
with open('/content/drive/MyDrive/labels.txt', 'r') as f:
    labels = f.read()

In [65]:
import numpy as np
with open('/content/drive/MyDrive/reviews.txt', 'r') as f:
    reviews = f.read()
with open('/content/drive/MyDrive/labels.txt', 'r') as f:
    labels = f.read()

In [66]:
print(reviews[:500])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which

positive
negative
po


In [67]:

from string import punctuation

# remove punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')

all_text = ' '.join(reviews_split)

# create a list of all words
all_words = all_text.split()


In [68]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [69]:
from collections import Counter

# Build a dictionary that maps indices to review lengths
counts = Counter(all_words)

review_lens = Counter([len(x.split()) for x in reviews_split])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [70]:
print('Number of reviews before removing outliers: ', len(reviews_split))

non_zero_idx = [ii for ii, review in enumerate(reviews_split) if len(review.split()) != 0]

reviews_split = [reviews_split[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_split))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


# ***Using a Pre-Trained Embedding Layer***

In [71]:
# Load Word2Vec
from gensim.models import KeyedVectors

# Creating the model
embed_lookup = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300-SLIM.bin',
                                                 binary=True)


In [72]:
pretrained_words = []
for index, word in enumerate(embed_lookup.index_to_key):
    pretrained_words.append(word)

In [73]:
row_idx = 1

# get word/embedding in that row
word = pretrained_words[row_idx]
embedding = embed_lookup[word]

# vocab and embedding info
print("Size of Vocab: {}\n".format(len(pretrained_words)))
print('Word in vocab: {}\n'.format(word))
print('Length of embedding: {}\n'.format(len(embedding)))
#print('Associated embedding: \n', embedding)

Size of Vocab: 299567

Word in vocab: for

Length of embedding: 300



Cosine Similarity

In [74]:
find_similar_to = 'fabulous'

print('Similar words to '+find_similar_to+': \n')

# Find similar words, using cosine similarity
for similar_word in embed_lookup.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.3f}".format(
        similar_word[0], similar_word[1]
    ))


Similar words to fabulous: 

Word: wonderful, Similarity: 0.761
Word: fantastic, Similarity: 0.761
Word: marvelous, Similarity: 0.730
Word: gorgeous, Similarity: 0.714
Word: lovely, Similarity: 0.713
Word: terrific, Similarity: 0.694
Word: amazing, Similarity: 0.693
Word: beautiful, Similarity: 0.670
Word: magnificent, Similarity: 0.667
Word: splendid, Similarity: 0.645


Tokenize reviews

In [75]:
def tokenize_all_reviews(embed_lookup, reviews_split):
    reviews_words = [review.split() for review in reviews_split]

    tokenized_reviews = []
    for review in reviews_words:
        ints = []
        for word in review:
            try:
                idx = embed_lookup.key_to_index[word]
            except:
                idx = 0
            ints.append(idx)
        tokenized_reviews.append(ints)

    return tokenized_reviews


In [76]:
tokenized_reviews = tokenize_all_reviews(embed_lookup, reviews_split)

In [77]:
print(tokenized_reviews[0])

[0, 137, 3, 0, 11620, 3799, 13, 1215, 10, 9, 194, 54, 12, 73, 61, 685, 41, 183, 243, 129, 12, 1663, 119, 72, 0, 9, 2989, 7334, 242, 159, 0, 453, 2, 0, 137, 1239, 19951, 3, 141, 1980, 0, 1898, 55, 3, 1663, 9, 11124, 0, 3857, 6663, 9, 20401, 295, 28, 45, 148, 157, 102, 27, 15452, 1663, 30714, 9, 65172, 0, 9, 844, 737, 47, 6585, 159, 0, 9, 668, 4365, 1003, 0, 27, 295, 56, 4365, 622, 9, 3832, 0, 43, 0, 897, 3187, 907, 0, 5396, 113, 9, 183, 4365, 1009, 3165, 10, 137, 0, 3288, 296, 10314, 4365, 6638, 213, 0, 8810, 40, 0, 116, 1663, 897, 2059, 0, 0, 137, 4365, 830, 2, 124, 2216, 0, 119, 782, 144, 2, 0, 137, 3, 330, 23046, 78, 0, 16915, 2, 13, 85275, 7451]


Padding sequences

In [78]:
def pad_features(tokenized_reviews, seq_length):
    features = np.zeros((len(tokenized_reviews), seq_length), dtype=int)
    for i, row in enumerate(tokenized_reviews):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features

In [79]:
seq_length = 200

features = pad_features(tokenized_reviews, seq_length=seq_length)
assert len(features)==len(tokenized_reviews), "Features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

print(features[:10,:20])

[[     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [ 16483     26      0     12 106210      0   1698     22     37     24
     432      1     72     30    275      0    303      0    162    126]
 [  1935   1326     12      0   1403     60   3921   2019      3   4809
      36      6   3172   7184    129   7951      0   2180   6098 166268]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0 

# ***Training, Validation, and Test Data***

In [80]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


DataLoaders and Batching

In [81]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# shuffling and batching data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [82]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,  1239,   554,   196],
        [    0,     0,     0,  ...,   119,   222,    90],
        [    0,     0,     0,  ...,    55,    25,  1083],
        ...,
        [  986,  4365,     8,  ...,     0,   183,    26],
        [18099,  4365,  2547,  ...,    40,    13,     3],
        [    0,     0,     0,  ...,    15, 82750,   708]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0])


Sentiment Network with PyTorch

In [83]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [84]:
import torch.nn as nn

class SentimentRNN(nn.Module):
  def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
    super(SentimentRNN, self).__init__()
    self.output_size = output_size;
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)

    self.dropout = nn.Dropout(0.3)

    self.fc = nn.Linear(hidden_dim, embedding_dim, output_size)
    self.sig = nn.Sigmoid()

  def forward(self,x, hidden):
    batch_size = x.size(0)
    x = x.long()
    embeds=self.embedding(x)
    lstm_out, hidden = self.lstm(embeds, hidden)
    #neww
    lstm_out = lstm_out[:, -1, :]

    out= self.dropout(lstm_out)
    out = self.fc(out)[:, 0]
    sig_out = self.sig(out)
    return sig_out, hidden

  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data

    if (train_on_gpu):
       hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
       weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
    else:
       hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
       weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

    return hidden

In [85]:
vocab_size = len(pretrained_words)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(299568, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=400, bias=True)
  (sig): Sigmoid()
)


Training

In [86]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [87]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.635671... Val Loss: 0.672261
Epoch: 1/4... Step: 200... Loss: 0.560111... Val Loss: 0.601486
Epoch: 1/4... Step: 300... Loss: 0.559800... Val Loss: 0.581099
Epoch: 1/4... Step: 400... Loss: 0.620388... Val Loss: 0.623393
Epoch: 2/4... Step: 500... Loss: 0.598536... Val Loss: 0.576957
Epoch: 2/4... Step: 600... Loss: 0.476709... Val Loss: 0.511161
Epoch: 2/4... Step: 700... Loss: 0.386443... Val Loss: 0.457597
Epoch: 2/4... Step: 800... Loss: 0.373941... Val Loss: 0.441280
Epoch: 3/4... Step: 900... Loss: 0.422151... Val Loss: 0.483860
Epoch: 3/4... Step: 1000... Loss: 0.219372... Val Loss: 0.455632
Epoch: 3/4... Step: 1100... Loss: 0.392883... Val Loss: 0.477250
Epoch: 3/4... Step: 1200... Loss: 0.462948... Val Loss: 0.446629
Epoch: 4/4... Step: 1300... Loss: 0.238527... Val Loss: 0.488366
Epoch: 4/4... Step: 1400... Loss: 0.155791... Val Loss: 0.468464
Epoch: 4/4... Step: 1500... Loss: 0.350579... Val Loss: 0.482890
Epoch: 4/4... Step: 1600... Loss: 

Testing

In [88]:
test_losses = []
num_correct = 0
# Replace net3 with net
net.eval()
# Initialize hidden state before the loop
h = net.init_hidden(batch_size)
for inputs, labels in test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # Creating new variables for the hidden state
    h = tuple([each.data for each in h])
    # Replace net3 with net and pass hidden state
    output, h = net(inputs, h)

    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    pred = torch.round(output.squeeze())

    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


print("Test loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.438
Test accuracy: 0.812


Inference on a test review

In [89]:
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

In [90]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower()
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()

    # Create vocab_to_int if it doesn't exist
    global vocab_to_int # Declare vocab_to_int as global to modify it
    if 'vocab_to_int' not in globals():
        vocab_to_int = {word: index for index, word in enumerate(pretrained_words)}

    test_ints = [vocab_to_int.get(word, 0) for word in test_words] # Correct get usage
    return test_ints # Return the tokenized review



In [91]:
# test sequence padding
seq_length= 200
test_inits = [tokenize_review(test_review_neg)] # Assign the tokenized review to test_inits
feature = pad_features(test_inits,seq_length) # Now test_inits is defined
print(features)

[[    0     0     0 ...    13 85275  7451]
 [    0     0     0 ...    14   441  4699]
 [16483    26     0 ...   726     6     0]
 ...
 [   73  2419     2 ...   716    36   117]
 [    0     0     0 ...    61   125    15]
 [    0     0     0 ...    25     3  5635]]


In [92]:
# test conversion to tensor and pass into your model
feature_tesnsor = torch.from_numpy(feature)
print(feature_tesnsor.size())

torch.Size([1, 200])


In [93]:
def predict(net, test_review, sequence_length=200):


    net3.eval()
    print(test_review)
    test_ints = tokenize_review(embed_lookup, test_review)

    seq_length=sequence_length
    features = pad_features([test_ints], seq_length)

    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)

    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()

    output = net3(feature_tensor)

    pred = torch.round(output.squeeze())
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))

    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [94]:
def predict (net, test_review, sequence_length=200):
  net.eval()
  # tikenize rewiew
  test_ints = tokenize_review( test_review)

  seq_length=sequence_length
  features = pad_features([test_ints], seq_length)
  feature_tensor = torch.from_numpy(features)
  batch_size = feature_tensor.size(0)
  h = net.init_hidden(batch_size)
  if(train_on_gpu):
    feature_tensor = feature_tensor.cuda()
  output, h = net(feature_tensor, h)
  pred = torch.round(output.squeeze())
  print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
  if(pred.item()==1):
    print("Positive review detected!")
  else:
    print("Negative review detected.")

In [95]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so great. I loved it!'

In [96]:
# call function
seq_length=200
predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.991529
Positive review detected!


Test on pos/neg reviews

In [97]:
seq_length=200

In [98]:
test_review_pos = 'This movie had the best acting and the dialogue was so great. I loved it.'

predict( net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.991529
Positive review detected!
